## Prerequisites
This notebook requires an experimental version of the cmdty-storage package only published on Test PyPI, plus the packages curves for smoothing the forward curve, and ipysheet which provides the input and output grids.

`
pip install -i https://test.pypi.org/simple/ cmdty-storage==0.1.0a17
pip install curves
pip install ipysheet
`

If this notebook has been downloaded standalone, i.e. without downloading the whole storage repo,
it requires the file [storage_gui.py](https://github.com/cmdty/storage/blob/master/samples/python/storage_gui.py) be in the same directory.


In [1]:
%matplotlib inline
import storage_gui as sg
sg.display_gui()

### To Do
* Buttons to clear data:
    * Forward curve.
    * Ratches.
* Import of data clears existing data first:
    * Forward curve.
    * Ratches.
* Optimise performance of data import (very slow at moment due to display updating on each cell change).
* Sort out truncation of labels.
* Interest rate curve, not flat rate.
* Ability to select granularity.
* Display simulation results in charts/grids.
    

In [2]:
sg.test_data_btn()

Button(description='Populate Test Data', style=ButtonStyle())

In [3]:
def create_fwd_input(dates, prices):
    dates_cells = sg.ips.Cell(
        value=dates,
        row_start=0,
        row_end=len(dates) - 1,
        column_start=0,
        column_end=0,
        type='date',
        date_format='YYYY-MM-DD',
        squeeze_row=False,
        squeeze_column=True
    )
    prices_cells = sg.ips.Cell(
            value=prices,
            row_start=0,
            row_end=len(prices) - 1,
            column_start=1,
            column_end=1,
            type='numeric',
            numeric_format='0.00',
            squeeze_row=False,
            squeeze_column=True
        )
    cells = [dates_cells, prices_cells]
    return sg.ips.Sheet(rows=len(dates), columns=2, cells=cells, row_headers=False,
                       column_headers=['fwd_start', 'price'])

dates = ['2021-01-14', '2021-02-01', '2021-03-01', None, None]
prices = [25.6, 43.56, 38.77, None, None]
fwd_sheet = create_fwd_input(dates, prices)
display(fwd_sheet)

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, date_format='YYYY-MM-DD', row_end=4, row_start=0, s…

In [4]:
sg.str_to_bool

<function storage_gui.str_to_bool(bool_text: str) -> bool>